# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096966


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:06<01:43,  3.44s/it]

Rendering models:   9%|▉         | 3/32 [00:07<01:11,  2.45s/it]

Rendering models:  12%|█▎        | 4/32 [00:07<00:49,  1.77s/it]

Rendering models:  16%|█▌        | 5/32 [00:07<00:34,  1.28s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:11<00:00,  4.81it/s]

Rendering models:  94%|█████████▍| 30/32 [00:11<00:00,  5.01it/s]

Rendering models:  97%|█████████▋| 31/32 [00:11<00:00,  5.34it/s]

Rendering models: 100%|██████████| 32/32 [00:12<00:00,  3.46it/s]

ElisabethB                              0.004597
not-logged-in-3b0b929f4475fd276561      0.001427
not-logged-in-3b0b929f4475fd276561      0.001864
martianmax                              0.000600
DevionJ                                 0.013972
jnarayanbvg                             0.000511
not-logged-in-c6917d9b19f41e6c2995      0.011213
awright5                                0.003399
thiagoxxxx                              0.000542
framosuw                                0.000509
not-logged-in-f7e2260ed4a9517a3e4a      0.121451
Kamphuisjes                             0.000523
CScampstudent                           0.104911
not-logged-in-ebe11e1994328fe49a89      0.076875
Galaxy-one                              0.000956
sn346808                                0.002666
not-logged-in-f7e2260ed4a9517a3e4a      0.188329
LeeJoseph                               0.003423
Donihee                                 0.001790
BottledFarts                            0.003788
not-logged-in-3505b6

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())